## 4.1.1 build model from block

In [1]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

2.0.0


In [2]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()    # Flatten层将除第一维（batch_size）以外的维度展平
        self.dense1 = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)

    def call(self, inputs):         
        x = self.flatten(inputs)   
        x = self.dense1(x)    
        output = self.dense2(x)     
        return output

In [3]:
X = tf.random.uniform((2,20))
net = MLP()
net(X)

<tf.Tensor: id=62, shape=(2, 10), dtype=float32, numpy=
array([[ 0.15637134,  0.14062534, -0.11187253, -0.13151687,  0.12066578,
         0.15376692,  0.03429577,  0.07023033, -0.12030508, -0.38496107],
       [-0.02877349,  0.1088542 , -0.20668823,  0.08241277,  0.06292161,
         0.25310248,  0.04884301,  0.27015388, -0.13183925, -0.23431192]],
      dtype=float32)>

## 4.1.2 Sequential

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

model(X)

<tf.Tensor: id=117, shape=(2, 10), dtype=float32, numpy=
array([[-0.42563885, -0.11981717,  0.0838763 ,  0.04553887,  0.09710997,
         0.16843301,  0.15290505, -0.00364013, -0.13743742, -0.36868355],
       [-0.37125233, -0.18243487,  0.24916942, -0.04006755,  0.06090571,
         0.05331742,  0.24555533, -0.03183865, -0.10122052, -0.11752242]],
      dtype=float32)>

## 4.1.3 build complex model

In [5]:
class FancyMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.rand_weight = tf.constant(
            tf.random.uniform((20,20)))
        self.dense = tf.keras.layers.Dense(units=20, activation=tf.nn.relu)

    def call(self, inputs):         
        x = self.flatten(inputs)   
        x = tf.nn.relu(tf.matmul(x, self.rand_weight) + 1)
        x = self.dense(x)    
        while tf.norm(x) > 1:
            x /= 2
        if tf.norm(x) < 0.8:
            x *= 10
        return tf.reduce_sum(x)

In [6]:
net = FancyMLP()
net(X)

<tf.Tensor: id=220, shape=(), dtype=float32, numpy=24.381481>

In [7]:
class NestMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.net = tf.keras.Sequential()
        self.net.add(tf.keras.layers.Flatten())
        self.net.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
        self.net.add(tf.keras.layers.Dense(32, activation=tf.nn.relu))
        self.dense = tf.keras.layers.Dense(units=16, activation=tf.nn.relu)

    
    def call(self, inputs):         
        return self.dense(self.net(inputs))

net = tf.keras.Sequential()
net.add(NestMLP())
net.add(tf.keras.layers.Dense(20))
net.add(FancyMLP())

net(X)

<tf.Tensor: id=403, shape=(), dtype=float32, numpy=3.2303767>